In [34]:

import firebase_admin
from firebase_admin import credentials, firestore

from google.cloud.firestore_v1.base_query import FieldFilter


service_account_path = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/atopile-880ca67acfe2.json"

if not firebase_admin._apps:  # Check if already initialized to prevent reinitialization
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

db = firestore.client()
_components_db = db.collection(
    "components"
)

In [109]:
from pint import UnitRegistry
import firebase_admin
from firebase_admin import firestore

ureg = UnitRegistry()


IGNORED_PROPERTIES = ['designator_prefix', 'mpn']

def get_component(req_json):
    # Sanitize 'type' input
    component_type = req_json.get("type", "").lower()
    if not component_type:
        raise ValueError("Component type is required.")

    physical_filters = {}
    property_filters = {"type": component_type}

    for key, value in req_json.items():
        if key in IGNORED_PROPERTIES or key in ["type", "package"]:
            continue  # Skip ignored and separately handled properties

        # Sanitize string inputs
        if isinstance(value, str):
            value = value.lower()

        if isinstance(value, dict) and "min_val" in value and "max_val" in value:
            base_min_val = convert_to_base_units(value["min_val"], value["unit"])
            base_max_val = convert_to_base_units(value["max_val"], value["unit"])
            physical_filters[key] = (base_min_val, base_max_val)
        else:
            property_filters[key] = value

    components = query_components(component_type, physical_filters, property_filters)

    if not components:
        return "No components found"

    # Select the best component
    return select_best_component(components)

def query_components(component_type, physical_filters=None, property_filters=None):
    query = db.collection("updated_fets").where("type", "==", component_type)

    # Apply property filters
    for prop, value in property_filters.items():
        if prop != "type":  # 'type' is already included in the query
            query = query.where(prop, "==", value)

    results = query.stream()
    components = [doc.to_dict() for doc in results]

    # Apply physical filters
    if physical_filters:
        components = [comp for comp in components if all(
            physical_filters[prop][0] <= comp.get(prop, {}).get('max_val', float('inf')) and
            physical_filters[prop][1] >= comp.get(prop, {}).get('min_val', 0)
            for prop in physical_filters
        )]

    return components

def convert_to_base_units(value, unit):
    quantity = ureg.Quantity(value, unit)
    base_unit = ureg.Quantity(1, unit).to_base_units().units
    return quantity.to(base_unit).magnitude

def select_best_component(components):
    in_stock_components = [comp for comp in components if comp.get("stock", 0) > 100]
    if in_stock_components:
        components = in_stock_components

    basic_parts = [comp for comp in components if comp.get("basic_part", False)]
    if basic_parts:
        components = basic_parts

    return min(components, key=lambda comp: comp.get("price_usd", float('inf')), default=None)

# Example usage
req_json = {
    "type": "capacitor",
    "value": {"unit": "microfarads", "min_val": 1, "max_val": 2},
    "package": "0402"
}

best_component = get_component(req_json)
print(best_component)


No components found


In [86]:
#

{'designator_prefix': 'C', 'mpn': 'generic_capacitor', 'type': 'capacitor', 'value': {'unit': 'microfarad', 'min_val': 0.9, 'max_val': 1.1, 'nominal': 1.0, 'tolerance':
                    0.10000000000000003, 'tolerance_pct': 10.000000000000004}, 'package': '0603'}  

get_component(query)

{'description': 'Multilayer Ceramic Capacitors MLCC - SMD/SMT 1uF 16V 0805 RoHS',
 'type': 'capacitor',
 'uuid': 'ato-16bfca99-6168-40fc-bcf8-35067a7ab6eb',
 'value': {'min_val': 9e-07,
  'nominal': 1e-06,
  'tolerance': 1.0000000000000005e-07,
  'unit': 'farad',
  'max_val': 1.1e-06,
  'tolerance_pct': 10.0},
 'stock': nan,
 'code': nan,
 'voltage': {'min_val': 0, 'unit': 'volt', 'max_val': 16.0},
 'price_usd': nan,
 'footprint': {'kicad': 'C0805'},
 'mpn': 'CL21F105ZOCNNNC',
 'datasheet': nan,
 'category': 'Multilayer Ceramic Capacitors MLCC - SMD/SMT',
 'lcsc_id': 'C50683',
 'package': '0805',
 'footprint_data': {'kicad': 'standard_library'},
 'dielectric': nan,
 'basic_part': False}

In [112]:
query = {'designator_prefix': 'C',
        'mpn': 'generic_mosfet',
        'type': 'mosfet',
        'current_a': {"unit": "ampere",
                    "min_val": 20,
                    "max_val": 200,
                    "nominal": 1.0,
                    "tolerance":0.10000000000000003,
                    "tolerance_pct": 10.000000000000004},
}

get_component(query)

{'type': 'mosfet',
 'stock': 637,
 'on_resistance_ohms': {'min_val': 0, 'unit': 'ohm', 'max_val': 0.38},
 'drain_source_voltage_v': {'min_val': 0, 'unit': 'volt', 'max_val': 20},
 'price_usd': 0.0284,
 'current_a': {'min_val': 0, 'unit': 'ampere', 'max_val': 75.0},
 'mpn': 'CJ3134K',
 'datasheet': 'Download',
 'footprint': {'kicad': 'SOT-23-11667b30-255a-42eb-b1bd-920e981e5eaa.kicad_mod'},
 'power_dissipation_w': {'min_val': 0, 'unit': 'watt', 'max_val': 0},
 'lcsc_id': 'C110100',
 'package': 'SOT-23',
 'footprint_data': {'kicad': '\t(module easyeda2kicad:SOT-23-3_L2.9-W1.6-P1.90-LS2.8-BR (layer F.Cu) (tedit 5DC5F6A4)\n\t(attr smd)\n\t(fp_text reference REF** (at 0 -4.95) (layer F.SilkS)\n\t(effects (font (size 1 1) (thickness 0.15)))\n\t)\n\t(fp_text value SOT-23-3_L2.9-W1.6-P1.90-LS2.8-BR (at 0 4.95) (layer F.Fab)\n\t(effects (font (size 1 1) (thickness 0.15)))\n\t)\n\t(fp_text user %R (at 0 0) (layer F.Fab)\n\t(effects (font (size 1 1) (thickness 0.15)))\n\t)\n\t(fp_line (start 0.88

In [15]:
{'designator_prefix': 'C', 'mpn': 'generic_capacitor', 'type': 'Capacitor', 'value': '{"unit": "microfarad", "min_val": 0.9, "max_val": 1.1, "nominal": 1.0, "tolerance":
                    0.10000000000000003, "tolerance_pct": 10.000000000000004}', 'package': '0603'}  

{nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [65]:
import uuid


def process_component_data(component):
    new_component = {}

    for key, value in component.items():
        new_key = key.lower().replace(' ', '_')  # Convert key to lowercase and replace spaces with underscores

        # Skip unwanted fields
        if key in ['value_in_farads', 'value_in_ohms', 'Comment', 'min_value', 'max_value', 'tolerance', 'unit', 'value']:
            continue

        # Rename specific keys
        if key == 'MFR.Part #':
            new_key = 'mpn'
        elif key == 'LCSC Part #':
            new_key = 'lcsc_id'
        elif key == 'Price (USD)':
            new_key = 'price_usd'

        # Transform 'voltage' to a 'Physical' type
        if key == 'voltage' and isinstance(value, (int, float)):
            new_component['voltage'] = {'unit': 'volt', 'max_val': value, 'min_val': 0}
            continue

        # Convert 'basic-extended' to boolean 'basic_part'
        if key == 'basic-extended':
            new_key = 'basic_part'
            value = True if value == 'Basic' else False

        if value != float('nan'):  # Exclude fields with NaN values
            new_component[new_key] = value

        # Exclude fields with NaN values
        if value != float('nan'):
            new_component[new_key] = value

    # set footprint data to 'standard_library'
    new_component['footprint_data'] = {'kicad': 'standard_library'}

    # add a UUID
    new_component['uuid'] = f'ato-{uuid.uuid4()}'

    # Determine component type
    component_type = new_component.get('type')
    if isinstance(component_type, float) and component_type != component_type:  # Check for NaN
        category = new_component.get('category', '').lower()
        if 'multilayer ceramic capacitors mlcc - smd/smt' in category:
            component_type = 'capacitor'
        elif 'chip resistor - surface mount' in category:
            component_type = 'resistor'

        new_component['type'] = component_type

    # Set 'value' property with appropriate unit
    if isinstance(component_type, str):
        if component_type.lower() == 'capacitor':
            unit = 'farad'
        elif component_type.lower() == 'resistor':
            unit = 'ohm'
        else:
            unit = None  # Default unit or handle other types as needed


        if unit:
            # calcualte tolerance and nominal values from min and max values
            min_val = component.get('min_value', 0)
            max_val = component.get('max_value', 0)
            new_component['value'] = {
                'unit': unit,
                'min_val': min_val,
                'max_val': max_val,
                'nominal': (min_val + max_val) / 2,
                'tolerance': (max_val - min_val) / 2,
                'tolerance_pct': round((max_val - min_val) / (min_val + max_val) * 100, 2) if min_val + max_val != 0 else 0
            }
    # if it is a resistor or capacitor, return the component, otherwise return None

    return new_component


def migrate_data():
    components = db.collection('components').limit(10).stream()  # Limit to the first 10 components

    for component in components:
        processed_data = process_component_data(component.to_dict())
        document_id = processed_data['uuid']
        db.collection('component_data').document(document_id).set(processed_data)
        print(f"Processed and added: {document_id}")  # Optional: Print each document ID for review

# Run the migration
migrate_data()


Processed and added: ato-3faf9491-e334-405d-a96f-80deddcd5df5
Processed and added: ato-8dd9dd5b-5b5c-434a-9c1b-3b472420c9e7
Processed and added: ato-935be1d1-7d2a-470d-8fcd-3c5f247c062b
Processed and added: ato-67cb172e-9962-4895-b36a-594498c7059d
Processed and added: ato-faffc6a6-fbda-4e58-ab30-fff66219b999
Processed and added: ato-b00eaa86-aae6-4ea4-a060-9f722b2c77c4
Processed and added: ato-3210ed47-e96b-40c0-b06d-5eff3fd4ef77
Processed and added: ato-1009f374-d28f-4c54-8a30-0fff54a79b20
Processed and added: ato-24501ac5-dad0-45cf-93a4-9508f102f80d
Processed and added: ato-aa6eff70-5446-4d42-9c11-61551e754385
Processed and added: ato-9e4c1ead-8d57-47b0-9bc3-8559d7d32855
Processed and added: ato-528ac197-cfa9-40c2-be26-ffaf87e7528f
Processed and added: ato-731d1249-f8b2-4a5f-b4e8-c97b1c359327
Processed and added: ato-419b1231-193d-4c7a-a4b8-de01d42170ac
Processed and added: ato-e689e040-9c0b-4be9-a85a-ba9a6284e6ae
Processed and added: ato-762fe245-f55c-4c3e-a534-ce6ebe2e58f3
Processe

In [100]:
url = "https://get-component-atsuhzfd5a-uc.a.run.app"

query = {'designator_prefix': 'C',
        'mpn': 'generic_capacitor',
        'type': 'capacitor',
        'value': {"unit": "microfarad",
                    "min_val": 0.8,
                    "max_val": 3,
                    "nominal": 1.0,
                    "tolerance":0.10,
                    "tolerance_pct": 10.00},
}

import requests
import json

response = requests.post(url, json=query)

# print(response.json)
print(response.json())

{'Best Component': {'description': 'Multilayer Ceramic Capacitors MLCC - SMD/SMT 2.2uF 10V 0603 RoHS', 'type': 'capacitor', 'uuid': 'ato-053c5bb7-b75c-4d88-9be8-ca962f98974e', 'value': {'min_val': 1.98e-06, 'nominal': 2.2e-06, 'tolerance': 2.2e-07, 'unit': 'farad', 'max_val': 2.42e-06, 'tolerance_pct': 10.0}, 'stock': nan, 'code': nan, 'voltage': {'min_val': 0, 'unit': 'volt', 'max_val': 10.0}, 'price_usd': nan, 'footprint': {'kicad': 'C0603'}, 'mpn': 'CL10F225ZP8NNNC', 'datasheet': nan, 'category': 'Multilayer Ceramic Capacitors MLCC - SMD/SMT', 'lcsc_id': 'C163510', 'package': '0603', 'footprint_data': {'kicad': 'standard_library'}, 'dielectric': nan, 'basic_part': False}}


In [101]:
query = db.collection("mosfets").where


In [102]:
# convert the first 10 results to a list of dicts
results = [doc.to_dict() for doc in query.limit(10).stream()]

# print the first result
print(results[0])

{'Assembly Type': 'Wave SolderingA PCB assembly fixture is needed to protect and support this part during the assembly process.', 'value': '500V 28A 310W 155mΩ@10V,14A N Channel TO-3P  MOSFETs ROHS', 'type': 'mosfet', 'uuid': 'df8a910a-23f4-4511-8659-21f218537fac', 'Description': '500V 28A 310W 155mΩ@10V,14A N Channel TO-3P  MOSFETs ROHS', 'Price': [{'quantity': '1+', 'price': 5.085}, {'quantity': '10+', 'price': 4.449}, {'quantity': '30+', 'price': 3.7035}, {'quantity': '90+', 'price': 3.321}, {'quantity': '450+', 'price': 3.1455}, {'quantity': '900+', 'price': 3.066}], 'Stock': 0, 'CAD Model': 'PCB Footprint or Symbol', 'Datasheet': 'Download', 'current_A': 28.0, 'on_resistance_ohms': 0.155, 'Source': 'JLCPCB', 'Package': 'TO-3P-3', 'drain_source_voltage_V': 500, 'MFR.Part #': 'FDA28N50', 'footprint': {'kicad': 'TO-3P-3-df8a910a-23f4-4511-8659-21f218537fac.kicad_mod'}, 'Manufacturer': 'onsemi', 'power_dissipation_W': 310, 'LCSC Part #': 'C105685', 'polarity': 'N Channel', 'footprint_

In [142]:
import re

def parse_description(description):
     # Adjusted regular expressions to match voltage, current, power, and resistance
    voltage_pattern = r'(\d+\.?\d*?)V'
    current_pattern = r'(\d+\.?\d*?mA|\d+\.?\d*?A)\b'  # Adjusted to capture mA and A with decimals
    power_pattern = r'(\d+\.?\d*?m?W)\b'  # Adjusted to capture mW and W with decimals
    resistance_pattern = r'(\d+\.?\d*?m?Ω)\b'  # Adjusted to capture mΩ and Ω with decimals

    voltage = re.search(voltage_pattern, description)
    current = re.search(current_pattern, description)
    power = re.search(power_pattern, description)
    resistance = re.search(resistance_pattern, description)

    # Function to convert power, resistance, and current values to numerical values
    def convert_to_numeric(value):
        if 'm' in value:
            return float(value.replace('m', '').replace('W', '').replace('A', '').replace('Ω', '')) / 1000
        return float(value.replace('W', '').replace('A', '').replace('Ω', ''))

    extracted_values = {
        'voltage': convert_to_numeric(voltage.group(1)) if voltage else None,
        'current': convert_to_numeric(current.group(1)) if current else None,
        'power': convert_to_numeric(power.group(1)) if power else None,
        'resistance': convert_to_numeric(resistance.group(1)) if resistance else None
    }
    return extracted_values


def get_unit_for_key(key):
    units = {
    'current_a': 'ampere',
    'on_resistance_ohms': 'ohm',
    'drain_source_voltage_v': 'volt',
    'power_dissipation_w': 'watt'
    }
    return units.get(key.lower(), '')

def process_mosfet_data(mosfet):
    new_mosfet = {}


    for key, value in mosfet.items():
        new_key = key.lower().replace(' ', '_')  # Convert key to lowercase and replace spaces with underscores

        # Rename specific keys
        if key == 'MFR.Part #':
            new_key = 'mpn'
        elif key == 'LCSC Part #':
            new_key = 'lcsc_id'
        elif key == 'Price':
            # Set price_usd to the 1+ pricing
            new_key = 'price_usd'
            value = next((price['price'] for price in value if price['quantity'] == '1+'), None)
        elif key == 'Stock':
            new_key = 'stock'
        elif key == 'Package':
            new_key = 'package'
        elif key == 'Datasheet':
            new_key = 'datasheet'
        elif key == 'footprint_data':
            # Preserve footprint_data
            continue
        elif key in ['value', 'CAD Model', 'Source', 'Manufacturer', 'Assembly Type', 'uuid']:
            continue  # Skip these fields

        # Convert specific properties to 'Physical' type
        if key in ['current_A', 'on_resistance_ohms', 'drain_source_voltage_V', 'power_dissipation_W']:
            new_mosfet[new_key] = {'unit': get_unit_for_key(key), 'max_val': value, 'min_val': 0}
        else:
            new_mosfet[new_key] = value

    # Preserve footprint_data as is
    if 'footprint_data' in mosfet:
        new_mosfet['footprint_data'] = mosfet['footprint_data']

      # Parse description and update data
    description = mosfet.get('Description', '')
    parsed_data = parse_description(description)

    for key, value in parsed_data.items():
        if value is not None:
            # Define the corresponding field in new_mosfet based on the key
            new_key = {
                'voltage': 'drain_source_voltage_v',
                'current': 'current_a',
                'power': 'power_dissipation_w',
                'resistance': 'on_resistance_ohms'
            }.get(key, None)
            if new_key:
                # Create or update the corresponding field
                new_mosfet[new_key] = {'unit': get_unit_for_key(new_key), 'max_val': value, 'min_val': 0}

    # Preserve footprint_data as is
    if 'footprint_data' in mosfet:
        new_mosfet['footprint_data'] = mosfet['footprint_data']

    return new_mosfet

def migrate_mosfets():
    mosfets = db.collection('mosfets').stream()

    for mosfet in mosfets:
        try:
            processed_data = process_mosfet_data(mosfet.to_dict())
            document_id = f'ato-{uuid.uuid4()}'  # Generate a unique document ID
            db.collection('component_data').document(document_id).set(processed_data)
            print(f"Processed and added: {document_id}")
        except Exception as e:
            print(f"Error processing: {mosfet.id}")
            print(e)

# Run the migration
migrate_mosfets()


Processed and added: ato-9dea80dc-dc90-416c-827d-b49ef04ccb01
Processed and added: ato-c2049e33-192f-4929-a738-494ac837a3e4
Processed and added: ato-3af594fd-ba93-4aaf-abe4-1110a9b97e53
Processed and added: ato-c39a9466-86c9-4e66-93ef-895ee14918f0
Processed and added: ato-3fec11c3-55eb-4d61-a959-ae5554439a7c
Processed and added: ato-08fd644b-ade5-4422-9d13-e1181fc3e580
Processed and added: ato-1af77524-8df7-40bc-8e3f-7d17e9da58e1
Processed and added: ato-76659068-263a-4dc1-a29b-fe5a06716de2
Processed and added: ato-f78a5765-f6a8-4233-bf0a-786ebc651977
Processed and added: ato-f4ff7b2e-87b1-47b5-8b68-a5107c1be387
Processed and added: ato-f29ab946-d117-43e7-89b5-972e62d2c2fa
Processed and added: ato-fc26d54e-f351-4cc2-b27b-7a7cd90858fa
Processed and added: ato-e845fa1e-f982-4889-8ee7-b463c05625b0
Processed and added: ato-169a82d8-c331-4884-9253-34111b8e2636
Processed and added: ato-f648bf9f-198a-4b94-9de3-3be90462dc5f
Processed and added: ato-4feddaff-3542-49da-a93d-21c6e6e0e527
Processe

In [143]:
def delete_components(component_ids):
    for comp_id in component_ids:
        try:
            db.collection('component_data').document(comp_id).delete()
            print(f"Deleted component: {comp_id}")
        except Exception as e:
            print(f"Error deleting component {comp_id}: {e}")

# List of component IDs to remove
component_ids = [
"ato-9dea80dc-dc90-416c-827d-b49ef04ccb01",
"ato-c2049e33-192f-4929-a738-494ac837a3e4",
"ato-3af594fd-ba93-4aaf-abe4-1110a9b97e53",
"ato-c39a9466-86c9-4e66-93ef-895ee14918f0",
"ato-3fec11c3-55eb-4d61-a959-ae5554439a7c",
"ato-08fd644b-ade5-4422-9d13-e1181fc3e580",
"ato-1af77524-8df7-40bc-8e3f-7d17e9da58e1",
"ato-76659068-263a-4dc1-a29b-fe5a06716de2",
"ato-f78a5765-f6a8-4233-bf0a-786ebc651977",
"ato-f4ff7b2e-87b1-47b5-8b68-a5107c1be387",
"ato-f29ab946-d117-43e7-89b5-972e62d2c2fa",
"ato-fc26d54e-f351-4cc2-b27b-7a7cd90858fa",
"ato-e845fa1e-f982-4889-8ee7-b463c05625b0",
"ato-169a82d8-c331-4884-9253-34111b8e2636",
"ato-f648bf9f-198a-4b94-9de3-3be90462dc5f",
"ato-4feddaff-3542-49da-a93d-21c6e6e0e527",
"ato-8d4eadec-8603-440c-9149-0d4757b3712d",
"ato-da1805c7-f54d-40b0-8e6d-c40156165b71",
"ato-f71b8b5c-ef8c-4f11-8eba-f4b0ccc42511",
"ato-a52e5b08-aa44-4f15-8527-f0981d062d24"
]

# Run the deletion process
delete_components(component_ids)

Deleted component: ato-9dea80dc-dc90-416c-827d-b49ef04ccb01
Deleted component: ato-c2049e33-192f-4929-a738-494ac837a3e4
Deleted component: ato-3af594fd-ba93-4aaf-abe4-1110a9b97e53
Deleted component: ato-c39a9466-86c9-4e66-93ef-895ee14918f0
Deleted component: ato-3fec11c3-55eb-4d61-a959-ae5554439a7c
Deleted component: ato-08fd644b-ade5-4422-9d13-e1181fc3e580
Deleted component: ato-1af77524-8df7-40bc-8e3f-7d17e9da58e1
Deleted component: ato-76659068-263a-4dc1-a29b-fe5a06716de2
Deleted component: ato-f78a5765-f6a8-4233-bf0a-786ebc651977
Deleted component: ato-f4ff7b2e-87b1-47b5-8b68-a5107c1be387
Deleted component: ato-f29ab946-d117-43e7-89b5-972e62d2c2fa
Deleted component: ato-fc26d54e-f351-4cc2-b27b-7a7cd90858fa
Deleted component: ato-e845fa1e-f982-4889-8ee7-b463c05625b0
Deleted component: ato-169a82d8-c331-4884-9253-34111b8e2636
Deleted component: ato-f648bf9f-198a-4b94-9de3-3be90462dc5f
Deleted component: ato-4feddaff-3542-49da-a93d-21c6e6e0e527
Deleted component: ato-8d4eadec-8603-440

In [127]:
import re

description = "30V 5.8A 40mΩ@10V,5.8A 1.4W N Channel SOT-23(TO-236)  MOSFETs ROHS"
import re

def convert_to_base_unit(value, unit):
    if unit:
        if 'm' in unit:
            return float(value) / 1000  # Convert milli to base unit
        elif 'k' in unit:
            return float(value) * 1000  # Convert kilo to base unit
        else:
            return float(value)

description = "20V 0.75A 200mW 380mΩ@4.5V,650mA N Channel SOT-323(SC-70) MOSFETs ROHS"

# Regular expressions to match voltage, current, power, and resistance in the description
voltage_pattern = r'(\d+(\.\d+)?)(m|k)?(?=V)'
current_pattern = r'(\d+(\.\d+)?)(m|k)?(?=A)'
power_pattern = r'(\d+(\.\d+)?)(m|k)?(?=W)'
resistance_pattern = r'(\d+(\.\d+)?)(m|k)?(?=Ω)'

# Extract values using regular expressions
voltage_match = re.search(voltage_pattern, description)
current_match = re.search(current_pattern, description)
power_match = re.search(power_pattern, description)
resistance_match = re.search(resistance_pattern, description)

extracted_values = {
    'voltage': convert_to_base_unit(*voltage_match.groups()[:2]) if voltage_match else None,
    'current': convert_to_base_unit(*current_match.groups()[:2]) if current_match else None,
    'power': convert_to_base_unit(*power_match.groups()[:2]) if power_match else None,
    'resistance': convert_to_base_unit(*resistance_match.groups()[:2]) if resistance_match else None
}

print(extracted_values)


{'voltage': None, 'current': 0.75, 'power': None, 'resistance': None}


In [134]:
import re

def parse_description_final(description):
    # Adjusted regular expressions to match voltage, current, power, and resistance
    voltage_pattern = r'(\d+\.?\d*?)V'
    current_pattern = r'(\d+\.?\d*?mA|\d+\.?\d*?A)\b'  # Adjusted to capture mA and A with decimals
    power_pattern = r'(\d+\.?\d*?m?W)\b'  # Adjusted to capture mW and W with decimals
    resistance_pattern = r'(\d+\.?\d*?m?Ω)\b'  # Adjusted to capture mΩ and Ω with decimals

    voltage = re.search(voltage_pattern, description)
    current = re.search(current_pattern, description)
    power = re.search(power_pattern, description)
    resistance = re.search(resistance_pattern, description)

    # Function to convert power, resistance, and current values to numerical values
    def convert_to_numeric(value):
        if 'm' in value:
            return float(value.replace('m', '').replace('W', '').replace('A', '').replace('Ω', '')) / 1000
        return float(value.replace('W', '').replace('A', '').replace('Ω', ''))

    extracted_values = {
        'voltage': convert_to_numeric(voltage.group(1)) if voltage else None,
        'current': convert_to_numeric(current.group(1)) if current else None,
        'power': convert_to_numeric(power.group(1)) if power else None,
        'resistance': convert_to_numeric(resistance.group(1)) if resistance else None
    }
    return extracted_values

# Testing the updated function with a new string
test_string = "20V 0.75A 200mW 380mΩ@4.5V,650mA N Channel SOT-323(SC-70) MOSFETs ROHS"
parsed_test_string = parse_description_final(test_string)
parsed_test_string



{'voltage': 20.0, 'current': 0.75, 'power': 0.2, 'resistance': 0.38}